In [7]:
import logging
from xbrl.cache import HttpCache
from xbrl.instance import XbrlParser, XbrlInstance

In [8]:
# just to see which files are downloaded
logging.basicConfig(level=logging.INFO)

cache: HttpCache = HttpCache('./cache')
cache.set_headers({'From': 'YOUR@EMAIL.com', 'User-Agent': 'py-xbrl/2.1.0'})
parser = XbrlParser(cache)

schema_url = "https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/aapl-20210925.htm"
inst: XbrlInstance = parser.parse_instance(schema_url)

INFO:xbrl.helper.connection_manager:200 https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/aapl-20210925.htm
INFO:xbrl.helper.connection_manager:200 https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/aapl-20210925.xsd
INFO:xbrl.helper.connection_manager:200 https://xbrl.fasb.org/srt/2021/elts/srt-2021-01-31.xsd
INFO:xbrl.helper.connection_manager:200 http://www.xbrl.org/2003/xbrl-instance-2003-12-31.xsd
INFO:xbrl.helper.connection_manager:200 http://www.xbrl.org/2003/xbrl-linkbase-2003-12-31.xsd
INFO:xbrl.helper.connection_manager:200 http://www.xbrl.org/2003/xl-2003-12-31.xsd
INFO:xbrl.helper.connection_manager:200 http://www.xbrl.org/2003/xlink-2003-12-31.xsd
INFO:xbrl.helper.connection_manager:200 https://www.xbrl.org/dtr/type/2020-01-21/types.xsd
INFO:xbrl.helper.connection_manager:200 https://www.xbrl.org/2020/extensible-enumerations-2.0.xsd
INFO:xbrl.helper.connection_manager:200 http://www.xbrl.org/2005/xbrldt-2005.xsd
INFO:xbrl.helper.conne

In [11]:
import json

In [15]:
import pandas as pd

In [18]:
report = json.loads(inst.json(override_fact_ids=True))

In [64]:
def satz_erkenner(i, leerzeichen=3):
    if len(str(i).split(" ")) > leerzeichen:
        return True
    return False

In [63]:
for key, value in report["facts"].items():
    if "value" in value.keys():
        if isinstance(value["value"], str):
            string = report["facts"][key]["value"]
            if len(string.split(" ")) > 4:
                print(string)

Common Stock, $0.00001 par value per share
Summary of Significant Accounting Policies
Basis of Presentation and PreparationThe consolidated financial statements include the accounts of Apple Inc. and its wholly owned subsidiaries (collectively “Apple” or the “Company”). Intercompany accounts and transactions have been eliminated. In the opinion of the Company’s management, the consolidated financial statements reflect all adjustments, which are normal and recurring in nature, necessary for fair financial statement presentation. The preparation of these consolidated financial statements and accompanying notes in conformity with U.S. generally accepted accounting principles requires management to make estimates and assumptions that affect the amounts reported. Actual results could differ materially from those estimates. Certain prior period amounts in the consolidated financial statements and accompanying notes have been reclassified to conform to the current period’s presentation.
The C

In [60]:
value

{'value': 'Net sales for 2021, 2020 and 2019 and long-lived assets as of September\xa025, 2021 and September\xa026, 2020 were as follows (in millions):',
 'dimensions': {'concept': 'ScheduleOfRevenuesFromExternalCustomersAndLongLivedAssetsByGeographicalAreasTableTextBlock',
  'entity': '0000320193',
  'period': '2020-09-27/2021-09-25'}}

In [37]:
df = pd.json_normalize(report["facts"]).T

In [40]:
df = df.rename(columns={0:"value"})

In [65]:
df["is_sentence"] = df["value"].apply(lambda x: satz_erkenner(x, 4))

In [69]:
import re

In [73]:
pattern = r"(?<!\b(?:Mr|Mrs|Dr|Ms)\b)(?<!\b\w)\.\s"

In [90]:
sentences = df.query("is_sentence==True").reset_index(drop=True)["value"].apply(lambda x: x.split(r". ")).explode().reset_index(drop=True)

In [91]:
sentences = pd.DataFrame(sentences)

In [92]:
sentences["e_score"] = sentences["value"].apply(lambda x: len(x))
sentences["s_score"] = sentences["value"].apply(lambda x: len(x) * 2)
sentences["g_score"] = sentences["value"].apply(lambda x: len(x) * 3)

In [93]:
sentences

,value,e_score,s_score,g_score
0,"Common Stock, $0.00001 par value per share",42,84,126
1,Summary of Significant Accounting Policies,42,84,126
2,Basis of Presentation and PreparationThe conso...,108,216,324
3,and its wholly owned subsidiaries (collectivel...,73,146,219
4,Intercompany accounts and transactions have be...,59,118,177
...,...,...,...,...
84,Although the reportable segments provide simil...,276,552,828
85,The accounting policies of the various segment...,136,272,408
86,The following table shows information by repor...,530,1060,1590
87,A reconciliation of the Company’s segment oper...,370,740,1110
